# Run scGen

add class documentation

In [ ]:
# Importing necessary libraries
import pertpy as pp
import scanpy as sc
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append('./')
from scGen_class import scGenPerturbationAnalysis

In [ ]:
# Loading the dataset
covid_data = pp.dt.stephenson_2021_subsampled()
#covid_data = sc.read('data/stephenson_2021_subsampled.h5ad')

In [ ]:
# input parameters
condition_col= "time_after_LPS" #or "disease" in case COVID vs normal
ctrl_key = "nan"
stim_key = "10h" # early LPS "90m"
celltype_col = "author_cell_type" # "cell_type"
celltype_to_predict = "B_naive"
n_epochs = 20
n_comps = 50 # n_comps for PCA and distance metric calculation

In [ ]:
# filter the data set to have pairwise comparison 
LPSN_data = covid_data[((covid_data.obs["disease"] == "normal"))]
LPSN_data = LPSN_data[(LPSN_data.obs[condition_col].isin([ctrl_key, stim_key]))]

In [ ]:
#downsample the data set
sc.pp.subsample(LPSN_data, n_obs=3000)

In [ ]:
# Filter data for control and stimulated conditions
ctrl_data = LPSN_data[LPSN_data.obs[condition_col] == ctrl_key]
stim_data = LPSN_data[LPSN_data.obs[condition_col] == stim_key]

# Get unique cell types for each condition
ctrl_celltypes = set(ctrl_data.obs[celltype_col].unique())
stim_celltypes = set(stim_data.obs[celltype_col].unique())

# Find the intersection (cell types present in both conditions)
common_celltypes = list(ctrl_celltypes.intersection(stim_celltypes))

print(common_celltypes)
len(common_celltypes)

In [ ]:
combined_counts = LPSN_data.obs.groupby([celltype_col,condition_col]).size() 
combined_counts_df = combined_counts.reset_index(name='counts')
combined_counts_df

In [ ]:
# Initialize an empty list to store the results
results = []

for i in common_celltypes:

    celltype_to_predict = i

    # Instantiating the PerturbationAnalysis class with the loaded data
    analysis = scGenPerturbationAnalysis(LPSN_data)
    # Preprocessing the data
    analysis.preprocess_data(apply_log1p = True) #normalisation mandatory, and log1p optional... maybe test both
    analysis.prepare_training_set(condition_col, 
                                stim_key,
                                celltype_col,
                                celltype_to_predict)
    # Setting up AnnData for scGen
    analysis.setup_anndata(condition_col, 
                        celltype_col)
    analysis.train_model(max_epochs= n_epochs, batch_size=64)
    # Making predictions 
    analysis.make_prediction(
                            ctrl_key,
                            stim_key,
                            celltype_to_predict,
                            condition_col)
    # Evaluating the predictions
    analysis.evaluate_prediction(celltype_col,
                                celltype_to_predict,
                                condition_col,
                                ctrl_key,
                                stim_key)
    analysis.plot_mean_correlation(stim_key) #
    
    # Get R2 value
    r2_value = analysis.r2_value

    # Computing the distance metric
    metric = "edistance"
    analysis.compute_distance_metric(n_comps,
                                    metric,
                                    condition_col,
                                    stim_key,
                                    ctrl_key)
    # Get the perturbation score
    e_distance = analysis.perturbation_score

    metric = "mmd"
    analysis.compute_distance_metric(n_comps,
                                    metric,
                                    condition_col,
                                    stim_key,
                                    ctrl_key)
    # Get the perturbation score
    mmd = analysis.perturbation_score #maximum mean distance

    metric = "euclidean"
    analysis.compute_distance_metric(n_comps,
                                    metric,
                                    condition_col,
                                    stim_key,
                                    ctrl_key)
    # Get the perturbation score
    euclidean = analysis.perturbation_score #maximum mean distance

    results.append({
            "celltype_to_predict": celltype_to_predict,
            "r2_value": r2_value,
            "e_distance": e_distance,
            "mmd": mmd,
            "euclidean": euclidean


        })
    

In [ ]:

results_df = pd.DataFrame(results)
results_df.to_csv("results_table_stephenson_20epochs_lateLPS.csv", index=False)


In [ ]:
#evaluate results:
# Load the CSV file into a DataFrame
data = pd.read_csv("results_table_stephenson_20epochs_lateLPS.csv")

# Set up the plotting style and dimensions
sns.set(style="whitegrid")
fig, axes = plt.subplots(1, 4, figsize=(20, 6), sharey=True)

# Define metrics and their labels for x-axis
metrics = ["r2_value", "e_distance", "mmd", "euclidean"]
x_labels = ["R2 Value", "E Distance", "MMD", "Euclidean"]

# Create a bar plot for each metric
for i, metric in enumerate(metrics):
    sns.barplot(
        y="celltype_to_predict", x=metric, data=data, ax=axes[i], palette="viridis"
    )
    axes[i].set_xlabel(x_labels[i])

# Set common title and show plot
plt.suptitle("Metrics by Cell Type")
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()